In [1]:
%matplotlib inline
from datetime import timedelta, datetime
from time import sleep
from bot import run_strategy, prepareLiveData
from liveBot import trading_supervisor
from tusp import string2ts

In [21]:
class LiveBot:

    def __init__(self, pair, period, strategy, weight=1):
        
        self.pair = pair
        self.period = period
        self.strategy = strategy
        self.weight = weight
        # en estrategía de ML, porcentaje de datos que se utilizaran 
        # en el conjunto de datos de entrenamiento
        self.per = 0.95
        # en esta lista deben guardarse los nombres de todas las estrategias
        # de machine learning en strategy, para discriminar en base a estas
        # la cantindad de datos a traer 
        self.ml_strategies = ["ml_logreg", "ml_randfor", "ml_knn"]
        self.ml_strategy = False
        self.have_coin = False
        self.delta = timedelta(seconds = int(self.period))
        # máximo delay aceptado entre el tiempo actual y el del último dato
        self.delta2 = timedelta(seconds = 200)
        # desfase para en la mayoría de los casos no entrar al while
        self.delta3 = timedelta(seconds = 90)

        self.len_data = 0
        # dinero inicial con el que empieza el paperBot
        self.balance = []
        # definiendo el tiempo inicial de la consulta
        
        print "Creado live Bot con las siguientes características:\n"
        print "\tpair: {0}, period: {1}, strategy: {2}, weight: {3}\n".format(self.pair, self.period,
                                                                             self.strategy, self.weight)
     
    def prepareData(self):
        """Trae el dataframe y el objeto poloniex correspondiente"""
        
        # definiendo end como la hora local actual
        self.tf = datetime.now()
        # convirtiendola a formato unix time (es equivalente a UTC)
        self.end = string2ts(self.tf.strftime('%Y-%m-%d %H:%M:%S'))
        #start = end 

        # definiendo el tiempo inicial de la consulta 
        self.to = self.tf-self.delta*self.len_data
        self.start = string2ts(self.to.strftime('%Y-%m-%d %H:%M:%S'))

        # trayendo y preparando datos
        self.df, self.polo = prepareLiveData(pair=self.pair, start=self.start, end=self.end, period=int(self.period))
        
        # fecha del último dato disponible
        self.tt = self.df.index[-1]
        # delay entre el tiempo actual y el último dato disponible
        self.de = self.tf - self.tt


    def lenData(self):
        """Define la cantidad de datos a traer de acuerdo si la estrategia es 
        o no de machine learning
        """ 

        if self.strategy in self.ml_strategies:
            # para estrategias de machine learning se tomarán los últimos
            # 7000 datos
            self.len_data = 7000
            self.ml_strategy = True
        else: 
            # para estrategias diferentes a las de ML se toman los últimos
            # 50 datos
            self.len_data = 40

            
    def timeSincronizator(self):
        """Trae los datos (dataFrame y objeto polo) siempre y cuando el delay 
        entre el tiempo actual y el últimio dato en el dataFrame no sea mayor 
        a 200 segundos (delta2). Sincroniza el tiempo del bot"""
        
        # definiendo self.tf, self.end, self.to, self.start, self.tt, self.de
        self.prepareData()

        # sincronizando tiempo del bot
        if self.de > self.delta2:
            # mientras la diferencia entre el la hora del último precio de cierre y
            # y la hora actual sea mayor a 120s
            while self.de > self.delta2:
                prepareData()
                sys.stdout.write("\rSincronizando bot (delay máximo aceptado 200s, actual %ss). Esperando cierre de las %s. Último dato de cierre a las %s"%(str((de).seconds), str(tt+self.delta), str(tt)))
                sys.stdout.flush()
                sleep(5)
        
        print "\n"
    
    def stratBot(self, btc_balance = None):
        """Inicia el bot y lo mantiene en un cilo de periodo self.period"""
        
        # definiendo la self.len_data y self.ml_strategy
        self.lenData()
        
        while True:
            try:
                # Trayendo datos sincronizados (crea dataFrame, objeto polo y variables temporales)
                self.timeSincronizator()
                
                # corriendo estrategia
                self.w, self.market_return = run_strategy(self.strategy,self.df,self.pair,self.ml_strategy,self.per)
                
                self.signal, self.close = self.w["orders"][-1], self.pair, self.df["close"][-1]

                self.run_live_signal(btc_balance)
               
                # calibrando tiempo de espera de acuerdo a emisión de próximo dato
                to_sleep = self.tt+self.delta-self.tf+self.delta3
                # se recarga cada to_sleep segundos
                sleep(to_sleep.seconds)

            # Saliendo del programa        
            except KeyboardInterrupt:

                yn = raw_input("\n\n\tDo you want to quit (y/n)? ")
                if yn == "y" or yn =="Y" or yn =="yes" or yn =="YES":
                    print "\tSeleccionó salir. \n" 

                    # guardando el resumen del paper trading en un archivo de texto
                    with open("paper_resume_%s_%s_%s.txt"%(self.pair, self.strategy, self.period),"w") as paper_resume:
                        try:
                            print >> paper_resume, "\tBalance: %s"%(self.balance[-1])
                        except IndexError:
                            print "\tHasta pronto..."
                            sys.exit()
                        print >> paper_resume, "\tProfit: {}%".format(round((self.balance[-1]-self.balance[0])/balance[0]*100, 2))

                    print "\tBalance: %s"%(balance[-1])
                    print "\tProfit: {}%".format(round((balance[-1]-balance[0])/balance[0]*100,2))
                    print "\tHasta pronto..."
                    bot_off(polo, order)
                elif yn == "n" or yn == "N" or yn == "no" or yn == "NO":
                    print "\tSeleccionó seguir"
                    pass

    def run_live_signal(self, base_balance=None):
        """Ejecuta en el exchange, muestra en pantalla 
        y envía correo de acuerdo a la señal dada por la estrategia
        
        :type btc_balance: float
        :param btc_balance: bitcoins disponibles para comprar el par
        """
        
        polo = self.polo
        pair = self.pair
        balance = self.balance
        time = str(self.tt)
        w = self.w["w"]
        signal = self.signal
        close = self.close
        base_simbol = pair.split("_")[0]

        fee = 0.0025
        if base_balance == None:
            base_balance = float(polo.returnBalances()[pair.split("_")[0]])
        
        coin_balance = float(polo.returnBalances()[pair.split("_")[1]])
        balance.append(btc_balance + coin_balance*close)

        destinatarios = open("destinatarios.txt").read()

        # corroborando si poseemos o no el activo
        if coin_balance >= 0.00001: self.have_coin = True
        else: self.have_coin = False

        order = []
        
        # si no deberíamos poseer el activo
        if w == False:
            # si tenemos el activo
            if self.have_coin:
                # colocando orden de venta de todas las monedas que tenemos 
                # para el par deseado, al último precio de cierre.
                order = polo.sell(pair, close, coin_balance)
                base_balance = coin_balance*close*(1-fee)
                coin_balance = 0.0
                balance[-1] = base_balance
                print "\n\tEstrategia: ",self.strategy,"\n"
                print time, pair, close, signal," ->balance:",round(balance[-1], 6),"COIN",coin_balance,base_simbol+":",base_balance
                correo(signal, time, pair, close, coin_balance, base_balance, balance[-1], self.strategy, destinatarios)
            # si no lo tenemos
            else:
                print time, pair, close, "WAIT"," ->balance:",round(balance[-1], 6),base_simbol
        
        # si deberíamos poseer el activo
        elif signal == True:
            # y no lo tenemos
            if not self.have_coin:
                # colocando orden de compra de todas las monedas que tenemos 
                # para el par deseado, al último precio de cierre.
                try:
                    order = polo.buy(pair, close, (base_balance)/close)
                except:
                    print "Error al comprar"
                    print "have_coin",self.have_coin,"close",close,"btc_balance",base_balance
                coin_balance = (base_balance/close)*(1-fee)
                base_balance = 0.0
                balance[-1] = coin_balance*close
                print "\n\tEstrategia: ",self.strategy,"\n"
                print time, pair, close, signal," ->balance:",round(balance[-1], 6),"COIN:",coin_balance,base_simbol+":",base_balance
                correo(signal, time, pair, close, coin_balance, base_balance, balance[-1], self.strategy, destinatarios)
            # si lo tenemos
            else:
                #print "\nhave_coin: {}, not have_coin {}".format(have_coin,not have_coin)
                #print "No quizo comprar el berraco"
                print time, pair, close, "WAIT"," ->balance:",round(balance[-1], 6),base_simbol

            # supervisando que la extrategia no esté perdiendo más del 10%
            trading_supervisor(polo, balance, order)

        return have_coin, coin_balance, btc_balance, order

In [22]:
firstBot = LiveBot("BTC_DGB", "300", "ml_knn")

Creado live Bot con las siguientes características:

	pair: BTC_DGB, period: 300, strategy: ml_knn, weight: 1



In [20]:
firstBot.len_data = 500
firstBot.prepareData()
print firstBot.to

2018-02-03 19:32:10.218000


In [ ]:
class MultiLiveBot():
    
    def __init__(self):
        """Clase que administra el funcionamiento de varios bots a la vez
        bots = lista de python con los bots a administrar 
        """
        self.bot_loader()
        self.n_bots = len(self.bots)
    
    def bot_loader(self):
        """Carga los datos para crear los bots desde el archivo
        multi_bot.inp, cuyo formato por línea es:
        par, %, estrategia, periodo
        """
        bots = []
        
        # abriendo archivo como lista
        multi_file = open("multi_bot.inp").readlines()
        
        for line in multi_file:
            if line.strip("\n").strip(" ") != "":
                pair = line.split(",")[0].strip(" ")
                weight = line.split(",")[1].strip(" ")
                strategy = line.split(",")[2].strip(" ")
                period = line.split(",")[3].strip("\n").strip(" ")
                bots.append({"pair":pair, "weight":weight, "strategy":strategy, "period":period})
        
        self.bots = bots
        
        # Lista con todos los pares del MultiBot
        pairs = []
        for bot in bots:
            pairs.append(bot["pair"])
        
        self.pairs = pairs
    
    def fundTrackerConstructor(self):
        """Crea archivo fundTracker con su header."""
        
        bots = ""
        for pair in self.pairs:
            bots += pair+","
            
        with open("fundTracker.inp","w") as tracker_file:
            tracker_file.write("Fecha,"+bots.strip(",")+"\n")
        
    def fundTracker(self, fecha, balances):
        """Escribe la distribución de los fondos administrados por
        cada bot en el archivo fundTracker.inp, cuyo formato por linea es:
        Fecha, balance par_1, balance par_2, ..., balance par_n
        
        :param fecha: Fecha de la linea de balance a escribir
        :type fecha: string con formato YYYY-mm-dd hh:mm:ss
        
        :param balances: contiene el balance actual en cada par
        :type balances: lista de python. Los fondos en la lista deben
        seguir el orden de self.pairs"""
        
        # Se agrega linea al final del archivo con el último dato de los balances
        with open("fundTracker.inp","a") as tracker_file:
            tracker_file.write(fecha+","+",".join(balances)+"\n")
            
    
    def fundTrackerReader(self):
        """Lee los fondos administrados por cada bot. Esto es,
        lee el archivo fundTracker.inp"""
        
        # Se abre el archivo fundTracker.inp como lista 
        tracker_file = open("fundTracker.inp").readlines()
        
        # Agregamos todo menos el header
        tracker_file = tracker_file[1:]
        
        
        return tracker_file
            

In [ ]:
administrador = MultiLiveBot()
administrador.fundTrackerConstructor()
administrador.fundTracker("2018-01-29 17:09:00",["0.14","0.56","0.76"])

In [ ]:
administrador.bots

In [ ]:
a = open("fundTracker.inp").readlines()
print administrador.pairs
print a[1:]

In [ ]:
administrador.pairs